In [9]:
import wandb
wandb.login()

True

In [10]:
import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.7.0 True
11.0
GCC 7.3


In [11]:
# 모듈 import
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)

In [19]:
seed = 42

set_random_seed(seed, deterministic=False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [20]:
from mmcv import Config

# config file 들고오기
baseline_cfg_path = './configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)

In [21]:
model_name = 'faster_rcnn_x101_64x4d_fpn_1x_coco'
exp = '0'

# Folder to store model logs and weight files
job_folder = f'./working/exp[{exp}]_{model_name}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'sunhyuk'
wnb_project_name = 'cv-3-bitcoin'

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: ./working/exp[0]_faster_rcnn_x101_64x4d_fpn_1x_coco


In [22]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'cv_train_1.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'cv_val_1.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.seed = seed #~~~
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10 ##클래수 수

cfg.evaluation = dict(
    interval=2,
    save_best='auto',
    metric='bbox',
    classwise=True,
)
    
# cfg.optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001)
cfg.optimizer = dict(type='Adam', lr=8e-5)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=50,
    warmup_ratio=0.001,
    step=[8, 11]
)

cfg.runner.max_epochs = 12

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         interval=10,
         init_kwargs=dict(project=wnb_username,entity=wnb_project_name, name=f'[{exp}]_{model_name}')
        )
]
cfg.log_config.interval = 10
# cfg.checkpoint_config = 20
print(cfg.pretty_text)


model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://resnext101_64x4d'),
        groups=64,
        base_width=4),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
      

In [23]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

# dataset 확인
datasets[0]

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!



CocoDataset Train dataset with number of images 3677, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2993  | 1 [Paper]     | 4685  | 2 [Paper pack]  | 681   | 3 [Metal]   | 711   | 4 [Glass]    | 733   |
| 5 [Plastic]       | 2211  | 6 [Styrofoam] | 970   | 7 [Plastic bag] | 3808  | 8 [Battery] | 136   | 9 [Clothing] | 378   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [24]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-04-02 18:18:35,455 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_64x4d'}
2022-04-02 18:18:35,460 - mmcv - INFO - load model from: open-mmlab://resnext101_64x4d
2022-04-02 18:18:35,460 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_64x4d
2022-04-02 18:18:35,739 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-02 18:18:35,766 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-02 18:18:35,775 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-04-02 18:18:35,880 - mmcv - INFO - 
backbone.conv1.weight 

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg=cfg, distributed=False, validate=True)